In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

print("📦 Imports OK")

📦 Imports OK


## 1. Chargement des données

In [2]:
# Charger
df_capteurs = pd.read_csv('../data/processed/capteurs_cleaned.csv')
df_satellites = pd.read_csv('../data/processed/satellites_cleaned.csv')

print(f"✅ Capteurs: {len(df_capteurs):,} observations")
print(f"✅ Satellites: {len(df_satellites):,} images")

# Convertir dates
df_capteurs['date'] = pd.to_datetime(df_capteurs['date'])
df_satellites['date'] = pd.to_datetime(df_satellites['date'])

print(f"\nPériode capteurs: {df_capteurs['date'].min().date()} → {df_capteurs['date'].max().date()}")
print(f"Période satellites: {df_satellites['date'].min().date()} → {df_satellites['date'].max().date()}")

✅ Capteurs: 73,052 observations
✅ Satellites: 13,721 images

Période capteurs: 2007-10-29 → 2022-08-01
Période satellites: 2015-09-13 → 2022-07-31


## 2. Filtrage période Sentinel-2

In [3]:
# Filtrer capteurs pour période Sentinel-2 UNIQUEMENT
print(f"Avant filtrage: {len(df_capteurs):,}")

df_capteurs = df_capteurs[
    (df_capteurs['date'] >= '2015-06-23') &  # Lancement Sentinel-2
    (df_capteurs['date'] <= '2022-12-31')
]

print(f"Après filtrage: {len(df_capteurs):,}")
print(f"Réduction: {(1 - len(df_capteurs)/73052)*100:.1f}%")
print(f"\nNouvelle période: {df_capteurs['date'].min().date()} → {df_capteurs['date'].max().date()}")

Avant filtrage: 73,052
Après filtrage: 30,365
Réduction: 58.4%

Nouvelle période: 2015-06-29 → 2022-08-01


## 3. Identification stations communes

In [4]:
# Stations avec données
stations_capteurs = set(df_capteurs['station_id'].unique())
stations_satellites = set(df_satellites['station_id'].unique())
stations_communes = stations_capteurs & stations_satellites

print(f"Stations capteurs: {len(stations_capteurs)}")
print(f"Stations satellites: {len(stations_satellites)}")
print(f"Stations COMMUNES: {len(stations_communes)} ✅")

# Filtrer aux stations communes
df_capteurs = df_capteurs[df_capteurs['station_id'].isin(stations_communes)]
df_satellites = df_satellites[df_satellites['station_id'].isin(stations_communes)]

print(f"\nDonnées filtrées:")
print(f"  Capteurs: {len(df_capteurs):,} observations")
print(f"  Satellites: {len(df_satellites):,} images")

Stations capteurs: 2139
Stations satellites: 73
Stations COMMUNES: 73 ✅

Données filtrées:
  Capteurs: 5,470 observations
  Satellites: 13,721 images


## 4. Fusion avec fenêtre temporelle optimisée

In [5]:
def fusion_par_station(df_cap, df_sat, window_days=14):
    """
    Fusionne capteurs et satellites par station
    Pour chaque capteur, prend l'image satellite la plus proche dans ±window_days
    """
    results = []
    
    stations = df_cap['station_id'].unique()
    print(f"Fusion pour {len(stations)} stations avec fenêtre ±{window_days} jours...")
    
    for i, station_id in enumerate(stations):
        if (i+1) % 10 == 0:
            print(f"  Station {i+1}/{len(stations)}...")
        
        # Données station
        cap_station = df_cap[df_cap['station_id'] == station_id].copy()
        sat_station = df_sat[df_sat['station_id'] == station_id].copy()
        
        if len(sat_station) == 0:
            continue
        
        # Pour chaque mesure capteur
        for idx, cap_row in cap_station.iterrows():
            date_cap = cap_row['date']
            
            # Chercher satellites dans fenêtre ±window_days
            sat_window = sat_station[
                (sat_station['date'] >= date_cap - timedelta(days=window_days)) &
                (sat_station['date'] <= date_cap + timedelta(days=window_days))
            ]
            
            if len(sat_window) == 0:
                continue
            
            # Prendre l'image la plus proche
            sat_window = sat_window.copy()
            sat_window['days_diff'] = abs((sat_window['date'] - date_cap).dt.days)
            closest = sat_window.nsmallest(1, 'days_diff').iloc[0]
            
            # Fusionner
            merged_row = {
                'date_capteur': date_cap,
                'date_satellite': closest['date'],
                'days_diff': closest['days_diff'],
                'station_id': station_id,
                'latitude': cap_row['latitude'],
                'longitude': cap_row['longitude'],
                # Capteurs
                'pH': cap_row['pH'],
                'oxygene_dissous': cap_row['oxygene_dissous'],
                'COD': cap_row['COD'],
                'CODMn': cap_row['CODMn'],
                'NH4N': cap_row['NH4N'],
                'TPH': cap_row['TPH'],
                'DIP': cap_row['DIP'],
                'DIN': cap_row['DIN'],
                'quality_score': cap_row['quality_score'],
                # Satellites
                'NDWI': closest['NDWI'],
                'chlorophyll_index': closest['chlorophyll_index'],
                'turbidity_index': closest['turbidity_index']
            }
            results.append(merged_row)
    
    return pd.DataFrame(results)

# Test avec différentes fenêtres
print("\n🔄 TEST FENÊTRES TEMPORELLES:\n")

for window in [7, 14, 21, 30]:
    df_merged = fusion_par_station(df_capteurs, df_satellites, window_days=window)
    taux = (len(df_merged) / len(df_capteurs)) * 100
    print(f"  ±{window} jours: {len(df_merged):,} matches ({taux:.1f}%)")


🔄 TEST FENÊTRES TEMPORELLES:

Fusion pour 73 stations avec fenêtre ±7 jours...
  Station 10/73...
  Station 20/73...
  Station 30/73...
  Station 40/73...
  Station 50/73...
  Station 60/73...
  Station 70/73...
  ±7 jours: 223 matches (4.1%)
Fusion pour 73 stations avec fenêtre ±14 jours...
  Station 10/73...
  Station 20/73...
  Station 30/73...
  Station 40/73...
  Station 50/73...
  Station 60/73...
  Station 70/73...
  ±14 jours: 361 matches (6.6%)
Fusion pour 73 stations avec fenêtre ±21 jours...
  Station 10/73...
  Station 20/73...
  Station 30/73...
  Station 40/73...
  Station 50/73...
  Station 60/73...
  Station 70/73...
  ±21 jours: 481 matches (8.8%)
Fusion pour 73 stations avec fenêtre ±30 jours...
  Station 10/73...
  Station 20/73...
  Station 30/73...
  Station 40/73...
  Station 50/73...
  Station 60/73...
  Station 70/73...
  ±30 jours: 627 matches (11.5%)


## 5. Fusion finale avec fenêtre optimale

In [6]:
# Utiliser fenêtre ±30 jours pour MAXIMUM de données
print("🚀 Fusion finale avec fenêtre ±30 jours...\n")

df_merged = fusion_par_station(df_capteurs, df_satellites, window_days=30)

print(f"\n✅ RÉSULTATS FUSION:")
print(f"  Lignes fusionnées: {len(df_merged):,}")
print(f"  Stations: {df_merged['station_id'].nunique()}")
print(f"  Taux de match: {(len(df_merged)/len(df_capteurs))*100:.1f}%")
print(f"  Écart temporel moyen: {df_merged['days_diff'].mean():.1f} jours")
print(f"  Écart temporel médian: {df_merged['days_diff'].median():.1f} jours")
print(f"  Écart temporel max: {df_merged['days_diff'].max()} jours")

🚀 Fusion finale avec fenêtre ±30 jours...

Fusion pour 73 stations avec fenêtre ±30 jours...
  Station 10/73...
  Station 20/73...
  Station 30/73...
  Station 40/73...
  Station 50/73...
  Station 60/73...
  Station 70/73...

✅ RÉSULTATS FUSION:
  Lignes fusionnées: 627
  Stations: 69
  Taux de match: 11.5%
  Écart temporel moyen: 13.0 jours
  Écart temporel médian: 12.0 jours
  Écart temporel max: 30 jours


## 6. Analyse qualité fusion

In [ ]:
# Distribution écarts temporels
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df_merged['days_diff'], bins=30, edgecolor='black')
plt.xlabel('Écart temporel (jours)')
plt.ylabel('Fréquence')
plt.title('Distribution des écarts temporels')
plt.axvline(df_merged['days_diff'].mean(), color='red', linestyle='--', label=f'Moyenne: {df_merged["days_diff"].mean():.1f}j')
plt.legend()

plt.subplot(1, 2, 2)
matches_per_station = df_merged.groupby('station_id').size()
plt.hist(matches_per_station, bins=20, edgecolor='black')
plt.xlabel('Nombre de matches')
plt.ylabel('Nombre de stations')
plt.title('Matches par station')
plt.axvline(matches_per_station.mean(), color='red', linestyle='--', label=f'Moyenne: {matches_per_station.mean():.1f}')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Statistiques par station:")
print(matches_per_station.describe())

## 7. Vérification valeurs manquantes

In [7]:
print("🔍 Valeurs manquantes:\n")
missing = df_merged.isnull().sum()
missing_pct = (missing / len(df_merged)) * 100

missing_df = pd.DataFrame({
    'Missing': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing'] > 0])

if missing.sum() == 0:
    print("✅ Aucune valeur manquante!")

🔍 Valeurs manquantes:

Empty DataFrame
Columns: [Missing, Percentage]
Index: []
✅ Aucune valeur manquante!


## 8. Sauvegarde dataset fusionné

In [8]:
# Sauvegarder
output_file = '../data/processed/merged_dataset.csv'
df_merged.to_csv(output_file, index=False)

print(f"💾 Dataset fusionné sauvegardé: {output_file}")
print(f"\n📊 RÉSUMÉ FINAL:")
print(f"  • Observations: {len(df_merged):,}")
print(f"  • Stations: {df_merged['station_id'].nunique()}")
print(f"  • Features capteurs: 9 (pH, DO, COD, CODMn, NH4N, TPH, DIP, DIN, quality_score)")
print(f"  • Features satellites: 3 (NDWI, chlorophyll_index, turbidity_index)")
print(f"  • Total features: 12 + métadonnées")
print(f"\n✅ Dataset prêt pour entraînement modèle!")

# Aperçu
print(f"\n📋 Aperçu (10 premières lignes):\n")
df_merged.head(10)

💾 Dataset fusionné sauvegardé: ../data/processed/merged_dataset.csv

📊 RÉSUMÉ FINAL:
  • Observations: 627
  • Stations: 69
  • Features capteurs: 9 (pH, DO, COD, CODMn, NH4N, TPH, DIP, DIN, quality_score)
  • Features satellites: 3 (NDWI, chlorophyll_index, turbidity_index)
  • Total features: 12 + métadonnées

✅ Dataset prêt pour entraînement modèle!

📋 Aperçu (10 premières lignes):



,date_capteur,date_satellite,days_diff,station_id,latitude,longitude,pH,oxygene_dissous,COD,CODMn,NH4N,TPH,DIP,DIN,quality_score,NDWI,chlorophyll_index,turbidity_index
0,2018-12-17,2018-12-29,12,1,43.838487,80.666649,8.1,7.70,0.86,0.8,0.26,0.014,0.008,0.145,10,-0.086186,1.086643,481600.0
1,2018-12-24,2018-12-29,5,1,43.838487,80.666649,8.2,8.04,0.86,0.9,0.28,0.014,0.008,0.145,10,-0.086186,1.086643,481600.0
2,2018-10-29,2018-10-27,2,2,30.270320,114.599967,7.7,8.58,0.86,4.5,0.15,0.014,0.008,0.145,10,0.502313,0.726601,59000.0
3,2018-11-05,2018-10-27,9,2,30.270320,114.599967,7.6,8.66,0.86,3.4,0.22,0.014,0.008,0.145,10,0.502313,0.726601,59000.0
4,2018-11-12,2018-10-27,16,2,30.270320,114.599967,7.6,8.84,0.86,2.8,0.26,0.014,0.008,0.145,10,0.502313,0.726601,59000.0
5,2018-12-17,2018-12-16,1,2,30.270320,114.599967,7.8,10.90,0.86,2.3,0.21,0.014,0.008,0.145,10,0.360146,0.822581,122400.0
6,2018-12-24,2018-12-16,8,2,30.270320,114.599967,7.9,11.70,0.86,2.3,0.25,0.014,0.008,0.145,10,0.360146,0.822581,122400.0
7,2018-10-29,2018-10-27,2,4,33.264061,115.339077,8.0,5.30,0.86,5.1,0.30,0.014,0.008,0.145,7,0.070664,0.844000,42200.0
8,2018-11-05,2018-11-01,4,4,33.264061,115.339077,8.1,5.47,0.86,5.6,0.14,0.014,0.008,0.145,7,0.098814,0.856115,59500.0
9,2018-11-12,2018-11-01,11,4,33.264061,115.339077,8.1,5.32,0.86,5.5,0.37,0.014,0.008,0.145,7,0.098814,0.856115,59500.0
